## **Bot card game**
- Bot is a mediator between peoples' dms

In [1]:
import discord
from discord.ext import commands
import asyncio
import random
from datetime import datetime, timedelta
import pickle as pkl 
import statistics as stats
import numpy as np
# from pprint import pprint # for help reading long lists

bot = commands.Bot(command_prefix = ('>>', '<<', '>.<', '<>', '-->'), case_insensitive = True, help_command = None) 

In [2]:
def check_for_repeats(new_list, old_lists):
    """Checks if anything in the new list is in any of the old lists, and returns the repeating items, along with the old lists the repeating items were in."""
    repeaters = list() # a list of the people repeating across the old lists
    whole_old_lists = list() # a list of the old lists with the repeaters

    for thing in new_list:
        for old_list in old_lists:
            if thing in old_list:
                repeaters.append(thing)
                whole_old_lists.append(old_list) # a list of the old list with the player
    return [repeaters, whole_old_lists]

def get_card_game(player):
    """Gets the player's card game using the check_for_repeats function"""
    with open("card_games.pkl", 'rb') as f:
        games = pkl.load(f) # loading the list of game classes
    all_players = [game.member_list for game in games] # all the players in all the games
    check = check_for_repeats(new_list = [player], old_lists = all_players)
    
    if check[0]:
        the_game_players = check[1][0] # taking the first list with repeaters
        the_game_index = all_players.index(the_game_players) 
        the_game = games[the_game_index] # the players' game class
        return [the_game, games]
    else:
        return []
    
def get_mafia_game(player):
    """Gets the player's mafia game using the check_for_repeats function"""
    with open("mafia_games.pkl", 'rb') as f:
        games = pkl.load(f) # loading the list of game classes
    all_players = [game.member_list for game in games] # all the players in all the games
    check = check_for_repeats(new_list = [player], old_lists = all_players)
    
    if check[0]:
        the_game_players = check[1][0] # taking the first list with repeaters
        the_game_index = all_players.index(the_game_players) 
        the_game = games[the_game_index] # the players' game class
        return [the_game, games]
    else:
        return []

In [3]:
def str_cardeck():
    """Generates a standard card deck sans jokers"""
    deck = [[str(rank)+suit for rank in range(2,11)] + [rank+suit for rank in ['ace', 'q', 'k', 'j']] \
            for suit in [' hearts', ' clubs', ' spades', ' diamonds']] # getting all the suits here
    deck = [j for i in deck for j in i] # make j a string so we can display the cards without brackets
    return deck

def show(cards):
    # making them showable
    formatted = [f"`{card}`" for card in cards]
    cards_sendable = ' | '.join(formatted)

    # returning the nicely showable cards
    return cards_sendable

class card_game: # functionality of the game
    def __init__(self, member_list, deck, orig_channel, game_start_time): 
        self.deck = deck 
        self.member_list = member_list 
        self.orig_channel = orig_channel
#         self.rotation = 0
        self.game_start_time = game_start_time
        self.table = {}
        
    def play_card(self, member, card): # playing the card from the member
        whole_deck = self.deck
        member_cards = whole_deck[member] # the member's card
        nobody_cards = whole_deck['Nobody'] # the cards assigned to "nobody"
        
        # taking the card from the member and adding it to discard
        member_cards.remove(card)
        nobody_cards.append(card) 
        
        # manipulating the table since the card was played
        try:
            table = self.table
            if card in table[member]:
                table[member].remove(card)
            if not len(table[member]): # so we don't have empty table entries
                table.pop(member)
        except KeyError:
            pass
        return 
        
    def draw_card(self, member):
        whole_deck = self.deck
        member_cards = whole_deck[member] # the member's cards
        nobody_cards = whole_deck['Nobody'] # the cards assigned to "nobody"
        
        # drawing a random card from the discard
        drawn_card = np.random.choice(nobody_cards)
        
        # taking the card from the deck and giving it to the player
        member_cards.append(drawn_card)
        nobody_cards.remove(drawn_card)
        return
        
    def game_stats(self):
        whole_deck = self.deck
        all_members = list(whole_deck.keys())
        
        # making a dict of each member and the number of cards they have
        stats = {member : len(whole_deck[member]) for member in all_members} 
        return stats
    
    def member_leave(self, leaver):
        deck_type = type(self.deck)
        if deck_type == dict:
            whole_deck = self.deck
            member_lst = self.member_list
            
            # adding the leaver's cards to discard
            leaver_cards = whole_deck[leaver]
            whole_deck['Nobody'] += leaver_cards

            # taking the leaver out of the game cards
            whole_deck.pop(leaver)
            self.member_list.remove(leaver)
            return 
        else: # when the cards haven't been dealt yet
            self.member_list.remove(leaver)
            return
    
    def member_join(self, joiner):
        deck_type = type(self.deck)
        if deck_type == dict: # when the cards have been dealt
            self.deck[joiner] = [] # initializing the joiner's card list
            return 
        elif deck_type == list: 
            self.member_list.append(joiner)
            return
        
#     def rotate(self): # rotating through the people # not very useful
#         while self.rotation >= len(self.member_list):
#             self.rotation -= len(self.member_list) # returning to the first if we went through the loop already
#         turn_person = self.member_list[self.rotation] # the person whose turn it is
#         self.rotation += 1
#         return turn_person 
    
    def shuffle(self): # shuffling the cards
        input_list = self.deck # the card list
        randagram = list() # initializing an empty list for the shuffled deck
        while input_list:
            card = np.random.choice(input_list)
            randagram.append(card)
            input_list.remove(card)
        self.deck = randagram # setting the deck to the randagram
        return 
    
    def deal_n_cards(self, split_amt): 
        n = int(split_amt)
        
        # splitting the deck into sections of length n
        if n:
            split_deck = [self.deck[i:i+n] for i in range(0, len(self.deck), n)]
        else:
            split_deck = [[]] * len(self.member_list)
        
        # assigning each deck section to a member
        assigned_split_deck = dict(zip(self.member_list, split_deck)) # dictionary to each person and their card list
        
        # a list of the cards given
        assigned_cards_lists = list(assigned_split_deck.values()) # list of all the lists of cards given
        assigned_cards = [j for i in assigned_cards_lists for j in i]
        
        # giving the remaining cards into "Nobody"
        deck = self.deck
        remaining_cards = [card for card in deck if not card in assigned_cards]
        assigned_split_deck['Nobody'] = remaining_cards
        
        # changing the value of the game deck
        self.deck = assigned_split_deck
        return
    
    def reset_deck(self):
        deck = str_cardeck()
        self.deck = deck # deck is now an unshuffled list of cards
        return 
    
    def give_cards(self, member, add_amt): 
        deck = self.deck
        nobody_cards = deck['Nobody']
        
        for i in range(add_amt):
            if not nobody_cards: # avoiding an error instead of using try/except
                break # stop giving someone cards if there are no cards to give
            # taking a random card from the discard and giving it to the member
            random_card = np.random.choice(nobody_cards)
            deck[member].append(random_card)
            nobody_cards.remove(random_card)
        return
    
    def play_time(self):
        now = datetime.now()
        gametime = (now - self.game_start_time) / timedelta(minutes = 1)
        return gametime
    
    def remove_n_cards(self, n):
        for i in range(n):
            if not len(self.deck):
                break # not giving cards if there are no cards to give
            # removing a random card
            card = np.random.choice(self.deck)
            self.deck.remove(card)
        return
    
    def fish(self, fisher, fished, card_rank): # getting all the cards of a specific rank from someone
        deck = self.deck
        fished_deck = deck[fished]
        rank = card_rank.lower()
        
        # getting all the cards of a rank
        fished_cards = [card for card in fished_deck if card.split(' ')[0] == rank]
        
        # adding all the cards of that rank to the fisher's cards
        deck[fisher] += fished_cards
        
        # taking the cards from the person we fished from
        for card in fished_cards:
            deck[fished].remove(card)
        return len(fished_cards)
    
    def table_put(self, p_id, card):
        try:
            if card not in self.table[p_id]: # so someone doesn't add many of the same card
                self.table[p_id].append(card)
        except KeyError:
            self.table[p_id] = [card] # initialize the person to the table if they weren't already there
        return
    
    def untable(self, p_id, card):
        # finding the person in the table with the card
        for table in self.table:
            if card in self.table[table]:
                tpeep = table
                tpeep_tabled_cards = self.table[table]
                break
        
        # taking the card from the table and from the original person
        if tpeep != "Nobody": # we took the card from nobody to put it on the table
            self.deck[tpeep].remove(card) # taking the card from the original owner
        tpeep_tabled_cards.remove(card) # taking the card from the table
        
        # updating the table of the person the card was taken from
        if tpeep_tabled_cards: # making sure there are cards left on their section of the table
            self.table[tpeep] = tpeep_tabled_cards
        else: # don't save an empty spot for cards
            self.table.pop(tpeep)
        
        # adding the card to the untabler
        self.deck[p_id].append(card)
        return tpeep # returning the person the card was untabled from

In [4]:
class play_cards(commands.Cog):
    def __init__(self, bot):
        self.bot = bot
        
    @commands.command(name = 'mention_card_game', aliases = ['mcg'])
    async def mention_playing(self, ctx):
        """**Start a card game with whoever you mention**"""
        if isinstance(ctx.channel, discord.DMChannel):
            await ctx.send("Start a card game from a guild, silly goose! <:goose_joker:738196077047578695>")
            return
        
        # loading the list of game classes
        with open("card_games.pkl", 'rb') as f:
            games = pkl.load(f) 
            
        # the new players
        channelers = ctx.channel.members
        players = [user.id for user in channelers if user.mentioned_in(ctx.message) and not user.bot]
        if ctx.author.id not in players:
            players.append(ctx.author.id)
        
        # checking if anyone is already in a game
        preexisting_players = [game.member_list for game in games] # will be a list of lists of ids
        check = check_for_repeats(new_list = players, old_lists = preexisting_players)
        
        if check[0]:
            # reporting all the people already in a game
            for user in check[0]:
                user = bot.get_user(user)
                await ctx.send(f"**{user.mention}** is already in a card game :pensive:")
            await ctx.send("Only one card game at a time per person, silly goose! <:goose_joker:738196077047578695>")
        else: # when no game with people in the players was found
            # making a new card game class
            cd = str_cardeck()
            game_start_time = datetime.now()
            orig_channel = int(ctx.channel.id)
            cg = card_game(players, cd, orig_channel, game_start_time)
            games.append(cg) # appending it to the preexisting list 
            
            # saving all the games
            with open("card_games.pkl", 'wb') as f:
                pkl.dump(games, f)
                
            # letting everyone know they're in the game
            for player in players:
                player = bot.get_user(player)
                await player.send("Card game started! :tada:")
                
    @commands.command(name = 'react_card_game', aliases = ['rcg'])
    async def react_playing(self, ctx):
        """**Start a card game with whoever reacts to my message**"""
        if isinstance(ctx.channel, discord.DMChannel):
            await ctx.send("Start a card game from a guild, silly goose! <:goose_joker:738196077047578695>")
            return
        
        # loading the list of game classes
        with open("card_games.pkl", 'rb') as f:
            games = pkl.load(f) 
        
        # making the new player poll embed
        poll = discord.Embed(color = 0xb2a2c2, description = "React to this message with 🎫 if you want to join the card game")
        poll.set_footer(text = "Poll times out after 20 seconds of inactivity",
                        icon_url = 'https://media.discordapp.net/attachments/720681623577690176/734896506917617664/joker_card_3.png?width=919&height=919')
        
        # sending the poll and adding a reaction to it
        poll_msg = await ctx.send(embed = poll)
        await poll_msg.add_reaction('🎫')
        
        # making sure we're only resetting the timer for the right reaction
        def check(reaction, user):
            return reaction.message.id == poll_msg.id and not user.bot and reaction.emoji == '🎫'

        # checking for reactions unless twenty seconds pass
        while True:
            try:
                reaction, user = await bot.wait_for('reaction_add', check = check, timeout = 20)
            except asyncio.TimeoutError:
                break
        
        # getting the people who reacted with ticket
        players = []
        poll_msg = await ctx.channel.fetch_message(poll_msg.id)
        for reaction in poll_msg.reactions:
            if reaction.emoji == '🎫':
                users = await reaction.users().flatten()
                players = [user.id for user in users if not user.bot]
        
        # returning if there are no reactions
        if not players:
            await ctx.send("All poll, no reacts :("
                           + "\nSilly gooses! <:goose_joker:738196077047578695>")
            return
        
        # checking if anyone is already in a game
        preexisting_players = [game.member_list for game in games] # will be a list of lists of ids
        check = check_for_repeats(new_list = players, old_lists = preexisting_players)
        
        if check[0]:
            # reporting all the people already in a game
            for user in check[0]:
                user = bot.get_user(user)
                await ctx.send(f"**{user.mention}** is already in a card game :pensive:")
            await ctx.send("Only one card game at a time per person, silly goose! <:goose_joker:738196077047578695>")
        else: # when no game with people in the players was found
            # making a new card game class
            cd = str_cardeck()
            game_start_time = datetime.now()
            orig_channel = int(ctx.channel.id)
            cg = card_game(players, cd, orig_channel, game_start_time)
            games.append(cg) # appending it to the preexisting list 
            
            # saving all the games
            with open("card_games.pkl", 'wb') as f:
                pkl.dump(games, f)
                
            # letting everyone know they're in the game
            for player in players:
                player = bot.get_user(player)
                await player.send("Card game started! :tada:")
    
    @commands.command(name = 'voice_card_game', aliases = ['vcg'])
    async def voice_playing(self, ctx):
        """**Start a card game with whoever you're in a voice channel with**"""
        if isinstance(ctx.channel, discord.DMChannel):
            await ctx.send("Start a card game from a guild, silly goose! <:goose_joker:738196077047578695>")
            return
        
        # loading the list of game classes
        with open("card_games.pkl", 'rb') as f:
            games = pkl.load(f) 
            
        # a list of the voice members
        try:
            voice_users = ctx.author.voice.channel.members
            players = [int(user.id) for user in voice_users if not user.bot]
        except AttributeError:
            await ctx.send("You need to be in a voice channel to start a mafia with its members, silly goose! <:goose_joker:738196077047578695>")
            return
        
        # checking if anyone is already in a game
        preexisting_players = [game.member_list for game in games] # will be a list of lists of ids
        check = check_for_repeats(new_list = players, old_lists = preexisting_players)
        
        if check[0]:
            # reporting all the people already in a game
            for user in check[0]:
                user = bot.get_user(user)
                await ctx.send(f"**{user.mention}** is already in a card game :pensive:")
            await ctx.send("Only one card game at a time per person, silly goose! <:goose_joker:738196077047578695>")
        else: # when no game with people in the players was found
            # making a new card game class
            cd = str_cardeck()
            game_start_time = datetime.now()
            orig_channel = int(ctx.channel.id)
            cg = card_game(players, cd, orig_channel, game_start_time)
            games.append(cg) # appending it to the preexisting list 
            
            # saving all the games
            with open("card_games.pkl", 'wb') as f:
                pkl.dump(games, f)
                
            # letting everyone know they're in the game
            for player in players:
                player = bot.get_user(player)
                await player.send("Card game started! :tada:")
    
    @commands.command(name = 'shuffle', aliases = ['sh'])
    async def deck_shuffle(self, ctx):
        """Shuffle your undealt deck of cards"""
        game_things = get_card_game(ctx.author.id)
        
        # checking if the author is in a game
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")
            return
        
        the_game = game_things[0]
        
        # saving all the games in a variable for updating 
        all_games = game_things[1]
        game_index = all_games.index(the_game) 
        
        # checking if the deck has been dealt yet
        deck_type = type(the_game.deck)
        if deck_type != list:
            await ctx.send("The deck was dealt already, silly goose! <:goose_joker:738196077047578695>")
            return 
        
        # shuffling the card deck
        the_game.shuffle() 
        
        # updating the save
        all_games[game_index] = the_game
        with open("card_games.pkl", 'wb') as f:
            pkl.dump(all_games, f)
            
        # notifying the original channel to let everyone know the deck was shuffled
        orig_channel = bot.get_channel(the_game.orig_channel)
        await orig_channel.send("The deck was shuffled!")
            
    @commands.command(name = 'deal_cards_evenly', aliases = ['dce'])
    async def even_card_dealing(self, ctx):
        """Divide the cards among yourselves"""
        game_things = get_card_game(ctx.author.id)
        
        # checking if the author is in a game
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")
            return

        the_game = game_things[0]
        deck_type = type(the_game.deck)

        # saving all the games in a variable for updating later
        all_games = game_things[1]
        game_index = all_games.index(the_game)
       
        # checking the state of the deck
        if not the_game.deck:
            await ctx.send("The- the cards are gone, silly goose! <:goose_joker:738196077047578695>")
            return
        elif deck_type != list:
            await ctx.send("The deck was dealt already, silly goose! <:goose_joker:738196077047578695>")
            return
        
        # diving the cards by the biggest number we can
        n = int(len(the_game.deck)/len(the_game.member_list)) # normal rounding might lead to too big a number
        the_game.deal_n_cards(n)
        
        # saving the game
        all_games[game_index] = the_game
        with open("card_games.pkl", 'wb') as f:
            pkl.dump(all_games, f)
        
        # telling each of the players their cards
        deck = the_game.deck
        for player in the_game.member_list:
            cards_sendable = show(deck[player])
            embed = discord.Embed(color = 0xb2a2c2)
            if cards_sendable:
                embed.add_field(name = "Your cards:", value = cards_sendable)
            else:
                embed.add_field(name = "Your cards:", value = "<:no_cards:708091052145377403>")
            player = bot.get_user(player)
            await player.send(embed = embed) 
            
    @commands.command(name = 'deal_cards_numerically', aliases = ['dcn', 'dnc'])
    async def n_card_dealing(self, ctx):
        """Give each player the integer number of cards you say"""
        game_things = get_card_game(ctx.author.id)
        
        # checking if the author is in a game
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")
            return

        the_game = game_things[0]
        deck_type = type(the_game.deck)
        
        # saving all the games in a variable for updating later
        all_games = game_things[1]
        game_index = all_games.index(the_game)
        
        # all the numbers in the message
        nums = [num for num in ctx.message.content.split(' ') if num.isdigit()] 
        
        # making sure we can deal the cards by an amount
        if not the_game.deck:
            await ctx.send("The- the cards are gone, silly goose! <:goose_joker:738196077047578695>")
            return
        if deck_type == dict:
            await ctx.send("The deck was dealt already, silly goose! <:goose_joker:738196077047578695>")
            return
        if not nums:
            await ctx.send("You need an integer split amount for dealing cards numerically, silly goose! <:goose_joker:738196077047578695>")
            return

        the_game.deal_n_cards(int(nums[0])) # now the deck is a dict
        
        # saving the game
        all_games[game_index] = the_game
        with open("card_games.pkl", 'wb') as f:
            pkl.dump(all_games, f)
            
        # DMing each of the players their cards
        deck = the_game.deck
        for player in the_game.member_list:
            # making an embed to nicely show the cards
            cards_sendable = show(deck[player])
            embed = discord.Embed(color = 0xb2a2c2)
            if cards_sendable:
                embed.add_field(name = "Your cards:", value = cards_sendable)
            else:
                embed.add_field(name = "Your cards:", value = "<:no_cards:708091052145377403>")
            # sending the nice embed to its owner
            player = bot.get_user(player)
            await player.send(embed = embed)
    
    @commands.command(name = 'my_cards', aliases = ['mc'])
    async def getting_my_cards(self, ctx):
        """DMs you your cards"""
        game_things = get_card_game(ctx.author.id)
        
        # checking if the author is in a game
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")
            return

        the_game = game_things[0]
        deck_type = type(the_game.deck)
        
        # making sure the author has cards
        if deck_type == list:
            await ctx.send("Cards haven't been dealt yet, silly goose! <:goose_joker:738196077047578695>")
            return
        
        # making the author's cards showable
        deck = the_game.deck
        cards_sendable = show(deck[ctx.author.id])
        
        # making a nice embed to show the author's cards
        embed = discord.Embed(color = 0xb2a2c2)
        if cards_sendable:
            embed.add_field(name = "Your cards:", value = cards_sendable)
        else:
            embed.add_field(name = "Your cards:", value = "<:no_cards:708091052145377403>")
        #sending the embed of cards
        await ctx.author.send(embed = embed)
    
    @commands.command(name = 'play_card', aliases = ['pc'])
    async def playing_a_card(self, ctx):
        """Play a card by rank and suit"""
        game_things = get_card_game(ctx.author.id)
        
        # checking if the author is in a game
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")
            return

        the_game = game_things[0]
        deck_type = type(the_game.deck)
        author_id = ctx.author.id 
        
        # saving to save the game later
        all_games = game_things[1]
        game_index = all_games.index(the_game)
        
        # checking if cards were dealt
        if deck_type == list:
            await ctx.send("Cards haven't been dealt yet, silly goose! <:goose_joker:738196077047578695>")
            return
        
        msg = ctx.message.content.lower().split(' ')
        
        # getting the card rank #
        ranks = [str(num) for num in range(2, 11)] + ['ace', 'q', 'k', 'j']
        rank = [word for word in msg if word in ranks]
        
        # getting the suit of the card
        suits = ['hearts', 'clubs', 'spades', 'diamonds']
        suit = [word for word in msg if word in suits]
        
        if not suit or not rank:
            help_embed = discord.Embed(color = 0xb2a2c2, 
                                       title = "Playable ranks and suits", 
                                       description = "You need to name a rank and a suit according to my rules, silly goose! <:goose_joker:738196077047578695>")
            help_embed.add_field(name = "Playable ranks", value = "`" + "` `".join(ranks) + "`")
            help_embed.add_field(name = "Playable suits", value = "`" + "` `".join(suits) + "`")
            
            await ctx.send(embed = help_embed)
            return
        
        # making the card
        card_rank = rank[0]
        card_suit = " "+suit[0]
        the_card = card_rank+card_suit
        
        # checking if the author can play the card
        legal_cards = the_game.deck[author_id]
        if the_card not in legal_cards:
            await ctx.send("You need to have the card you're playing, silly goose! <:goose_joker:738196077047578695>")
            return
        
        # finally playing the card
        the_game.play_card(member = author_id, card = the_card)
        
        # saving the game
        all_games[game_index] = the_game
        with open("card_games.pkl", 'wb') as f:
            pkl.dump(all_games, f)
            
        # making the cards showable
        deck = the_game.deck
        cards_sendable = show(deck[ctx.author.id])
        
        # making the embed to show the cards
        embed = discord.Embed(color = 0xb2a2c2)
        if cards_sendable:
            embed.add_field(name = "Your cards:", value = cards_sendable)
        else:
            embed.add_field(name = "Your cards:", value = "<:no_cards:708091052145377403>")
            
        #sending the embed of cards
        await ctx.author.send(embed = embed)
        
        # notifying the original channel
        orig_channel = bot.get_channel(the_game.orig_channel)
        await orig_channel.send(f"{ctx.author.name} played a card")
                
    @commands.command(name = 'draw_card', aliases = ['dc'])
    async def drawing_card(self, ctx):
        """Gives you a card from the discard pile"""
        game_things = get_card_game(ctx.author.id)
        
        # checking if the author is in a game
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")
            return

        the_game = game_things[0]
        deck_type = type(the_game.deck)
        
        # all the games, for saving later
        all_games = game_things[1]
        game_index = all_games.index(the_game)
        
        # making sure we can give the person a card
        if deck_type == list:
            await ctx.send("Cards haven't been dealt yet, silly goose! <:goose_joker:738196077047578695>")
            return
        if not the_game.deck['Nobody']:
            await ctx.send("No cards to take from <:goose_joker:738196077047578695>")
            return

        the_game.draw_card(ctx.author.id)
        
        # saving the game
        all_games[game_index] = the_game
        with open("card_games.pkl", 'wb') as f:
            pkl.dump(all_games, f)
            
        # sending the author their cards now
        deck = the_game.deck
        cards_sendable = show(deck[ctx.author.id])
        
        embed = discord.Embed(color = 0xb2a2c2)
        embed.add_field(name = "Your cards:", value = cards_sendable)
        await ctx.author.send(embed = embed)
        
        # notifying the original channel
        orig_channel = bot.get_channel(the_game.orig_channel)
        await orig_channel.send(f"{ctx.author.name} drew a card")
                
    @commands.command(name = 'get_stats', aliases = ['gs'])
    async def getting_the_game_stats(self, ctx):
        """Tells you how many cards each player has, and how many cards are in the discard pile"""
        game_things = get_card_game(ctx.author.id)
        
        # checking if the author is in a game
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")
            return

        the_game = game_things[0]
        deck_type = type(the_game.deck)
        
        # checking the state of the deck
        if deck_type == list:
            await ctx.send("Cards haven't been dealt yet, silly goose! <:goose_joker:738196077047578695>")
            return

        stats_dict = the_game.game_stats()
        
        # making an embed for stats
        embed = discord.Embed(color = 0xb2a2c2)
        for stat_name in stats_dict:
            if type(stat_name) == str:
                embed.add_field(name = stat_name, value = stats_dict[stat_name])
            else:
                player = bot.get_user(stat_name)
                player_name = player.name
                embed.add_field(name = player_name, value = stats_dict[stat_name])
        # sending the stat embed to the context
        await ctx.send(embed = embed)
                
    @commands.command(name = 'leave_card_game', aliases = ['lcg'])
    async def leaving_card_game(self, ctx):
        """You leave the game (applicable in a DM to me or any other channel I'm in)"""
        game_things = get_card_game(ctx.author.id)
        
        # checking if the author is in a game
        if not game_things:
            await ctx.send("Oh, silly goose! <:goose_joker:738196077047578695> You can't leave a game you're not in!")
            return
        
        the_game = game_things[0]
        
        # saving to save the game
        all_games = game_things[1]
        game_index = all_games.index(the_game)

        the_game.member_leave(ctx.author.id)
        
        # saving the game
        all_games[game_index] = the_game
        with open("card_games.pkl", 'wb') as f:
            pkl.dump(all_games, f)
            
        # telling the author bye
        await ctx.author.send("Goodbye, silly goose! <:goose_joker:738196077047578695>")
        
        # notifying the original channel they left
        orig_channel = bot.get_channel(the_game.orig_channel)
        await orig_channel.send(f"{ctx.author.name} has left the game, the silly goose! <:goose_joker:738196077047578695>")
        
        # removing games with all but one player gone
        # this means no solo games that originally had more players
        if len(the_game.member_list) < 2:
            time_playing = the_game.play_time()
            orig_channel = bot.get_channel(the_game.orig_channel)
            
            # saving all games
            all_games.remove(the_game)
            with open("card_games.pkl", 'wb') as f:
                pkl.dump(all_games, f)  
                
            # notifying the original channel their game is over
            await orig_channel.send(f"Game over! Hope y'all silly gooses had fun playing for {int(time_playing)} minutes and {int(round(60*(time_playing%1), 0))} seconds!")
                
    @commands.command(name = 'join_card_game', aliases = ['jcg'])
    async def joining_game(self, ctx):
        """Mentioned people join the game of the first person mentioned"""
        async with ctx.typing():
            # checking that the person is added from a guild
            if isinstance(ctx.channel, discord.DMChannel):
                await ctx.send("Join a game from a guild, silly goose! <:goose_joker:738196077047578695>")
                return
                
            # getting the game by a mention
            # loading the games
            with open("card_games.pkl", 'rb') as f:
                games = pkl.load(f)
            # all the mentioned people
            mentioned = [int(user.id) for user in ctx.channel.members if user.mentioned_in(ctx.message) and not user.bot]
            # all the player lists in all the games
            all_players = [game.member_list for game in games]
            # calling the first person mentioned the original gamer
            player_in_the_game = mentioned[0]
            # presuming there's no game
            the_game = None
            # looping through all the player lists
            for player_list in all_players: 
                # checking if the original gamer is in the player list
                if player_in_the_game in player_list: # if the first person mentioned is in a game's member id list
                    # storing the game in question in a variable
                    game_index = all_players.index(player_list)
                    the_game = games[game_index] # the players' game class
                    
                    # saving the player list in a variable
                    game_players = player_list
                    break # stopping the loop after finding the game
                    
            # checking if there is a game
            if not the_game:
                await ctx.send("No game found for the first person you mentioned, silly goose! <:goose_joker:738196077047578695>")
                return
            
            # making sure players are added from the original channel
            orig_channel = bot.get_channel(the_game.orig_channel) # getting the original channel
            if ctx.channel != orig_channel:
                await ctx.send(f"Add people from {orig_channel.mention}, silly goose! <:goose_joker:738196077047578695>")
                return
            
            # making sure we're not adding the same person to the game they're already in
            mentioned.pop(0) 
            if not mentioned: 
                await ctx.send("Mention people to join the game, silly goose! <:goose_joker:738196077047578695>")
                return
            
            # going through the mentioned people (except the first)
            for joiner in mentioned:
                # making sure we don't add a person already in a game
                if joiner in game_players:
                    await ctx.send(f"Hey! {joiner.name} was already in the game, silly goose! <:goose_joker:738196077047578695>")
                else:
                    the_game.member_join(joiner)
                    await ctx.send(f"{joiner.mention} Welcome to the card game, silly goose! <:goose_joker:738196077047578695>")
            
            # saving the game 
            games[game_index] = the_game
            with open("card_games.pkl", 'wb') as f:
                pkl.dump(games, f)
                
    @commands.command(name = 'reset_deck', aliases = ['rd'])
    async def resetting_the_deck(self, ctx):
        """Returns the deck to an unshuffled/undealt card deck, and empties the table"""
        game_things = get_card_game(ctx.author.id)
        
        # checking there is a game
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")
            return

        the_game = game_things[0]
        
        # saving the games for updating later
        all_games = game_things[1]
        game_index = all_games.index(the_game)
        
        # resetting the deck and table
        the_game.reset_deck()
        the_game.table = {}
        
        # saves the games
        all_games[game_index] = the_game
        with open("card_games.pkl", 'wb') as f:
            pkl.dump(all_games, f)
            
        # notifying the original channel of the change
        orig_channel = bot.get_channel(the_game.orig_channel)
        await orig_channel.send("The deck has been reset. I suggest a shuffle, silly gooses! <:goose_joker:738196077047578695>")
                
    @commands.command(name = 'get_cards', aliases = ['gc'])
    async def give_person_cards(self, ctx):
        """Gives you the (integer) number of cards you say from the discard pile"""
        game_things = get_card_game(ctx.author.id)
        
        # checking if there is a game connected to the author
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")
            return

        the_game = game_things[0]
        deck_type = type(the_game.deck)
        
        # saving all the games to update later
        all_games = game_things[1]
        game_index = all_games.index(the_game)
        
        # making sure the deck is dealable
        if deck_type == list:
            await ctx.channel.send("Just deal cards, silly goose! <:goose_joker:738196077047578695>" 
                                   + "\n||You deal yourself no cards and then try ;)||")
            return

        if ctx.channel.id == the_game.orig_channel: # so people don't cheat in certain games 
            # taking the first integer in the message
            msg = ctx.message.content.split(' ')
            cards_amt = [int(i) for i in msg if i.isdigit()][0]
            
            the_game.give_cards(ctx.author.id, cards_amt)
            
            # saving the game
            all_games[game_index] = the_game
            with open("card_games.pkl", 'wb') as f:
                pkl.dump(all_games, f)
                
            # making the cards showable
            deck = the_game.deck
            cards_sendable = show(deck[ctx.author.id])
            
            # making an embed to show the cards
            embed = discord.Embed(color = 0xb2a2c2)
            if cards_sendable:
                embed.add_field(name = "Your cards:", value = cards_sendable)
            else: # sending an empty embed is weird
                embed.add_field(name = "Your cards:", value = "<:no_cards:708091052145377403>")
            await ctx.author.send(embed = embed)
            
            # notifying the original channel
            orig_channel = bot.get_channel(the_game.orig_channel)
            await orig_channel.send(f"{ctx.author.name} has taken {cards_amt} cards")
        else:
            orig_channel = bot.get_channel(the_game.orig_channel)
            await ctx.send(f"Add those cards from the {orig_channel.mention}, silly goose! <:goose_joker:738196077047578695>") # what a sneaky goose
    
    @commands.command(name = 'cards_play_time', aliases = ['cpt', 'cgt'])
    async def gameplay_time(self, ctx):
        """How long you've been playing cards"""
        game_things = get_card_game(ctx.author.id)
        if game_things:
            the_game = game_things[0]
            gameplay_time = the_game.play_time()
            
            # making a nice embed to send the playtime 
            embed = discord.Embed(color = 0xb2a2c2)
            embed.add_field(name = "Time playing:", value = f"{int(gameplay_time)} minutes and {int(round(60*(gameplay_time%1), 0))} seconds")
            await ctx.send(embed = embed)
        else:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")
    
    @commands.command(name = 'remove_n_cards', aliases = ['rnc'])
    async def removing_n_cards(self, ctx):
        """Takes out the integer number of cards you say from the undealt deck or the discard pile"""
        game_things = get_card_game(ctx.author.id)
        
        # checking if there is a game connected to the author
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")                
            return
        
        the_game = game_things[0]
        deck_type = type(the_game.deck)
        
        # saving to save the game later
        all_games = game_things[1]
        game_index = all_games.index(the_game)
        
        # getting the integer card to remove
        nums = [int(i) for i in ctx.message.content.split(' ') if i.isdigit()]
        if nums:
            remove_num = nums[0]
        else:
            await ctx.send("Say an integer number of cards for me to toss into the void, silly goose! <:goose_joker:738196077047578695>")
        
        # checking if the game deck is a list of cards
        if deck_type == list:
            the_game.remove_n_cards(remove_num)
            
            # saving the game
            all_games[game_index] = the_game
            with open("card_games.pkl", 'wb') as f:
                pkl.dump(all_games, f)
                
            # notifying the original channel
            orig_channel = bot.get_channel(the_game.orig_channel)
            await orig_channel.send(f"{remove_num} cards have been thrown into the void")
        # if the deck has been dealt
        elif deck_type == dict:
            # tossing cards from "Nobody" to the void
            for i in range(remove_num):
                if not len(the_game.deck["Nobody"]):
                    break
                # removing a random card
                card = np.random.choice(the_game.deck["Nobody"]) 
                the_game.deck["Nobody"].remove(card)
                
            # saving the game
            all_games[game_index] = the_game
            with open("card_games.pkl", 'wb') as f:
                pkl.dump(all_games, f)
                
            # notifying the original channel
            orig_channel = bot.get_channel(the_game.orig_channel)
            await orig_channel.send(f"{remove_num} cards have been thrown into the void")
            
    @commands.command(name = 'fish_cards', aliases = ['fish'])
    async def fishing_rank(self, ctx):
        """If you have a card of the rank, you take all the cards of that rank from the person you mention"""
        game_things = get_card_game(ctx.author.id)
        
        # checking if there is a game connected to the author
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")                
            return
        
        the_game = game_things[0]
        deck_type = type(the_game.deck)
        
        # saving all the games in a variable for saving the game
        all_games = game_things[1]
        game_index = all_games.index(the_game)
        
        # making sure the fishing is from the original channel
        if ctx.channel.id != the_game.orig_channel:
            orig_channel = bot.get_channel(the_game.orig_channel)
            await ctx.send(f"Fish cards from the {orig_channel.mention}, silly goose! <:goose_joker:738196077047578695>")
        
        # making sure people have cards
        elif deck_type == list:
            await ctx.send("Cards haven't been dealt yet, silly goose! <:goose_joker:738196077047578695>")
        elif deck_type == dict:
            # a list of card ranks in the author's hand
            ranks = [card.split(" ")[0] for card in the_game.deck[ctx.author.id]]
            # a list of ranks in the message
            msg = ctx.message.content.lower().split(' ')
            said_ranks = [word for word in msg if word in ranks]
            # the humans mentioned in the message
            members = ctx.channel.members
            fished = [user for user in members if user.mentioned_in(ctx.message) and not user.bot]
            
            # making sure we have fishing materials
            if not said_ranks:
                await ctx.send("Name a rank you have, silly goose! <:goose_joker:738196077047578695>")
                return
            if not fished:
                await ctx.send("Mention someone in the game to take that rank from, silly goose! <:goose_joker:738196077047578695>")
                return
            
            # getting the id of the fished player
            fished_pl = fished[0] # the fished player is the first person mentioned
            fished_id = int(fished_pl.id)
            
            # making sure the fished person is in the game too
            if fished_id not in the_game.member_list:
                await ctx.send(f"{fished_pl.name} isn't in your game, silly goose! <:goose_joker:738196077047578695>") # not mentioning them to be nice
            else:
                cards_taken = the_game.fish(fisher = ctx.author.id, fished = fished_id, card_rank = said_ranks[0])
                
                # making sure cards were taken from a person
                if not cards_taken: # fishing returns the number of cards taken
                    # giving the person a card from the discard pile
                    if not the_game.deck['Nobody']:
                        await ctx.send(f"No cards for you, {ctx.author.name}! <:goose_joker:738196077047578695>")
                    else:
                        the_game.draw_card(ctx.author)

                        all_games[game_index] = the_game
                        with open("card_games.pkl", 'wb') as f:
                            pkl.dump(all_games, f)
                        
                        # making a nice embed to update the author
                        embed = discord.Embed(color = 0xb2a2c2, description = f"You didn't get any cards from {fished.mention}, but you drew a card!")
                        embed.add_field(name = "Your cards:", value = ' '.join(the_game.deck[ctx.author.id]))
                        await ctx.author.send(embed = embed)
                        
                        # notifying about the draw
                        await ctx.send(f"{ctx.author.name} drew a card")
                else:
                    # making the player's cards showable
                    deck = the_game.deck
                    cards_sendable = show(deck[ctx.author.id])
                    
                    # making an embed to show the author their cards
                    embed = discord.Embed(color = 0xb2a2c2)
                    embed.add_field(name = "Your cards:", value = cards_sendable)
                    await ctx.author.send(embed = embed)
                    
                    # notifying about how the fish went
                    await ctx.send(f"{ctx.author.name} got {cards_taken} cards from {fished_pl.name}")
                
                # saving the game
                all_games[game_index] = the_game
                with open("card_games.pkl", 'wb') as f:
                    pkl.dump(all_games, f)
                    
    @commands.command(name = 'table_put', aliases = ['tp', 'tc'])
    async def table_putting(self, ctx):
        """Allows you to show cards you have in a table-like manner. Tabled cards can be taken by anyone"""
        game_things = get_card_game(ctx.author.id)
        
        # checking if there is a game connected to the author
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")
            return

        the_game = game_things[0]
        
        deck_type = type(the_game.deck)
        author_id = ctx.author.id
        
        # saving all the games in variable to save the game later
        all_games = game_things[1]
        game_index = all_games.index(the_game)
        
        msg = ctx.message.content.lower().split(' ')
        
        # getting the card rank #
        ranks = [str(num) for num in range(2, 11)] + ['ace', 'q', 'k', 'j']
        rank = [word for word in msg if word in ranks]
        
        # getting the suit of the card
        suits = ['hearts', 'clubs', 'spades', 'diamonds']
        suit = [word for word in msg if word in suits]
        
        if not suit or not rank:
            help_embed = discord.Embed(color = 0xb2a2c2, 
                                       title = "Playable ranks and suits", 
                                       description = "You need to name a rank and a suit according to my rules, silly goose! <:goose_joker:738196077047578695>")
            help_embed.add_field(name = "Playable ranks", value = "`" + "` `".join(ranks) + "`")
            help_embed.add_field(name = "Playable suits", value = "`" + "` `".join(suits) + "`")
            
            await ctx.send(embed = help_embed)
            return
        
        # making the card
        card_rank = rank[0]
        card_suit = suit[0]
        the_card = card_rank+" "+card_suit
        
        # checking if the author can play the card
        legal_cards = the_game.deck[author_id]
        if the_card not in legal_cards:
            await ctx.send("You need to have the card you're showing on the table, silly goose! <:goose_joker:738196077047578695>")
            return
        
        # checking if cards were dealt
        if deck_type == list:
            await ctx.send("Cards haven't been dealt yet, silly goose! <:goose_joker:738196077047578695>")
            return
        
        the_game.table_put(author_id, the_card)
        table = the_game.table
        
        # saving the game
        all_games[game_index] = the_game
        with open("card_games.pkl", 'wb') as f:
            pkl.dump(all_games, f)
            
        # making an embed to show the table
        table_embed = discord.Embed(color = 0xb2a2c2, title = "┳━┳ Table ┳━┳")
        for person in table:
            nice_cards = show(table[person])
            
            if type(person) == str:
                table_embed.add_field(name = "Nobody", value = nice_cards)
            else:
                table_embed.add_field(name = bot.get_user(person).name, value = nice_cards)
        # sending the tabled embed to the ctx
        await ctx.send(embed = table_embed)
    
    @commands.command(name = 'table_take', aliases = ['tt', 'untable'])
    async def untabling(self, ctx):
        """Take a tabled card into your hand"""
        game_things = get_card_game(ctx.author.id)
        
        # making sure the ctx author is in a game
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")
            return
        
        the_game = game_things[0]
        deck_type = type(the_game.deck)
        
        # saving this stuff in variables so we can save the game later
        all_games = game_things[1]
        game_index = all_games.index(the_game)
        
        # making sure people untable from the original channel
        if ctx.channel.id != the_game.orig_channel:
            orig_channel = bot.get_channel(the_game.orig_channel)
            await ctx.send(f"Take from the table in {orig_channel.mention}, silly goose! <:goose_joker:738196077047578695>")
            return
        
        # checking if the cards were dealt
        if deck_type == list:
            await ctx.send("Cards haven't been dealt yet, silly goose! <:goose_joker:738196077047578695>" 
                           + "\n||You could deal yourself no cards and try again ;)||")
            return
        
        msg = ctx.message.content.lower().split(' ')
        
        # getting the card rank #
        ranks = [str(num) for num in range(2, 11)] + ['ace', 'q', 'k', 'j']
        rank = [word for word in msg if word in ranks]
        
        # getting the suit of the card
        suits = ['hearts', 'clubs', 'spades', 'diamonds']
        suit = [word for word in msg if word in suits]
        
        if not suit or not rank:
            help_embed = discord.Embed(color = 0xb2a2c2, 
                                       title = "Playable ranks and suits", 
                                       description = "You need to name a rank and a suit according to my rules, silly goose! <:goose_joker:738196077047578695>")
            help_embed.add_field(name = "Playable ranks", value = "`" + "` `".join(ranks) + "`")
            help_embed.add_field(name = "Playable suits", value = "`" + "` `".join(suits) + "`")
            
            await ctx.send(embed = help_embed)
            return
        
        # making the card
        card_rank = rank[0]
        card_suit = suit[0]
        the_card = card_rank+" "+card_suit
        
        # checking if the author can play the card
        legal_cards = [j for i in the_game.table.values() for j in i]
        if the_card not in legal_cards:
            await ctx.send("The card needs to be on the table, silly goose! <:goose_joker:738196077047578695>")
            return
        
        else:
            untabled_person = the_game.untable(ctx.author.id, the_card)
            
            # saving the game
            all_games[game_index] = the_game
            with open("card_games.pkl", 'wb') as f:
                pkl.dump(all_games, f)
            
            # making an embed to show the table now
            table = the_game.table
            table_embed = discord.Embed(color = 0xb2a2c2, title = "┳━┳ Table ┳━┳")
            for person in table:
                showable = show(table[person])
                
                if type(person) == str:
                    table_embed.add_field(name = "Nobody", value = showable)
                    continue
                table_embed.add_field(name = bot.get_user(person).name, value = showable)
            # sharing the table embed to the ctx (which will always be the game channel but still)
            await ctx.send(embed = table_embed)
            
            # formatting the cards
            deck = the_game.deck
            cards_sendable = show(deck[ctx.author.id])
            
            # making the embed to show the cards
            embed = discord.Embed(color = 0xb2a2c2)
            if cards_sendable:
                embed.add_field(name = "Your cards:", value = cards_sendable)
            else:
                embed.add_field(name = "Your cards:", value = "<:no_cards:708091052145377403>")
            await ctx.author.send(embed = embed)
            
            # sending the original owner of the cards their cards now if they're not the message author or nobody
            if untabled_person not in [ctx.author.id, "Nobody"]:
                # making the card list presentable
                deck = the_game.deck
                cards_sendable = show(deck[untabled_person])
                
                # making an embed to present the cards
                embed = discord.Embed(color = 0xb2a2c2)
                if cards_sendable:
                    embed.add_field(name = "Your cards:", value = cards_sendable)
                else:
                    embed.add_field(name = "Your cards:", value = "<:no_cards:708091052145377403>")
                
                # sending the embed to the untabled user
                untabled_user = bot.get_user(untabled_person)
                await untabled_user.send(embed = embed)
    
    @commands.command(name = 'table_random', aliases = ['tr', 'rt'])
    async def table_random_card(self, ctx):
        """Displays a random, unassigned card **without saving it to the table's "Nobody" section**"""
        game_things = get_card_game(ctx.author.id)
        
        # making sure the ctx author is in a game
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")
            return
        
        the_game = game_things[0]
        deck_type = type(the_game.deck)
        
        # checking if the ctx channel is the original channel
        if ctx.channel.id != the_game.orig_channel:
            orig_channel = bot.get_channel(the_game.orig_channel)
            await ctx.send(f"Show a deck card from the {orig_channel.mention}, silly goose! <:goose_joker:738196077047578695>")
            return
        
        # checking if the deck was dealt yet
        if deck_type == dict:
            nobody_cards = the_game.deck["Nobody"]
            
            # checking thare are cards to display from
            if not nobody_cards:
                await ctx.send("No cards to display from, silly goose! <:goose_joker:738196077047578695>")
                return
            # picking a random card and adding it to the "Nobody" part of the table
            card = np.random.choice(nobody_cards)
            try:
                the_game.table["Nobody"].append(card)
            except KeyError:
                the_game.table["Nobody"] = [card]
            
            # making an embed to show the table
            table = the_game.table
            table_embed = discord.Embed(color = 0xb2a2c2, title = "┳━┳ Table ┳━┳")
            for person in table:
                showable = show(table[person])
                
                if type(person) == str:
                    table_embed.add_field(name = person, value = showable)
                    continue
                table_embed.add_field(name = bot.get_user(person).name, value = showable)
            # sending the table embed
            await ctx.send(embed = table_embed)
        # if the deck wasn't dealt yet
        elif deck_type == list:
            game_deck = the_game.deck
            
            # checking if there are cards to display from
            if not game_deck:
                await ctx.send("No cards to display from, silly goose! <:goose_joker:738196077047578695>")
                return
            
            # picking a random card and adding it to the table
            random_card = np.random.choice(game_deck) 
            nadie_table = list(the_game.table.values()) # there can't be sections for specific people; cards haven't been dealt
            nadie_table.append(random_card)
            
            showable = show(nadie_table)
            
            # making the embed to display a random card
            table_embed = discord.Embed(color = 0xb2a2c2, title = "┳━┳ Table ┳━┳")
            table_embed.add_field(name = "Nobody", value = showable)
            # sending the table embed
            await ctx.send(embed = table_embed)
            
    @commands.command(name = 'table_nobody', aliases = ['tn', 'nt'])
    async def table_random_card_to_nadie(self, ctx):
        """Displays a random, unassigned card **and saves it to the table's "Nobody" section, removing it from the deck**"""
        game_things = get_card_game(ctx.author.id)
        
        # making sure there is a game with the ctx author
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")
            return
        
        the_game = game_things[0]
        deck_type = type(the_game.deck)
        
        # saving all the games in a variable for updating the deck later
        all_games = game_things[1]
        game_index = all_games.index(the_game) 
        
        # making sure cards are being displayed from the original channel
        if ctx.channel.id != the_game.orig_channel:
            orig_channel = bot.get_channel(the_game.orig_channel)
            await ctx.send(f"Show a deck card from the {orig_channel.mention}, silly goose! <:goose_joker:738196077047578695>")
            return
        
        # if cards have been dealt
        if deck_type == dict:
            nobody_cards = the_game.deck["Nobody"]
            
            if not nobody_cards:
                await ctx.send("No cards to add to table from, silly goose! <:goose_joker:738196077047578695>")
                return
        
            table = the_game.table
            # choosing a random card from the unassigned cards
            card = np.random.choice(nobody_cards)
            # adding the random card to the nobody section
            try:
                table["Nobody"].append(card)
                # removing the random card from nobody
                nobody_cards.remove(card)
            except KeyError:
                table["Nobody"] = [card]
                # removing the random card from nobody
                nobody_cards.remove(card)
            
            # making an embed to show the table
            table_embed = discord.Embed(color = 0xb2a2c2, title = "┳━┳ Table ┳━┳")
            
            for person in table:
                showable = show(table[person])
                
                if type(person) == str:
                    table_embed.add_field(name = "Nobody", value = showable)
                    continue
                table_embed.add_field(name = bot.get_user(person).name, value = showable)
            # sending the table embed
            await ctx.send(embed = table_embed)
        # when no cards have been dealt
        elif deck_type == list:
            game_deck = the_game.deck
            
            if not game_deck:
                await ctx.send("No cards to add to nobody from, silly goose! <:goose_joker:738196077047578695>")
                return
            
            table = the_game.table
            # choosing a random card
            card = np.random.choice(game_deck)
            # adding the card to the nobody section of the table and taking it from the deck
            try:
                the_game.table["Nobody"].append(card)
                game_deck.remove(card)
            except KeyError:
                table["Nobody"] = [card]
                game_deck.remove(card)
            
            # making the cards nice
            showable = show(table["Nobody"]) 
            
            # making an embed and showing the updated table (we don't need more fields because cards haven't been dealt yet)
            table_embed = discord.Embed(color = 0xb2a2c2, title = "┳━┳ Table ┳━┳")
            table_embed.add_field(name = "Nobody", value = showable)
            # sending the table embed
            await ctx.send(embed = table_embed)
        
        # saving the game
        all_games[game_index] = the_game
        with open("card_games.pkl", 'wb') as f:
            pkl.dump(all_games, f)
        
    @commands.command(name = 'table', aliases = ['t'])
    async def show_table(self, ctx):
        """Shows the currently tabled cards"""
        game_things = get_card_game(ctx.author.id)
        
        # making sure the author is in a card game
        if not game_things:
            await ctx.send("You're not playing a card game, silly goose! <:goose_joker:738196077047578695>")
            return
        
        the_game = game_things[0]
        
        deck_type = type(the_game.deck)
        
        # saving all the games in a variable for saving the game later
        all_games = game_things[1]
        game_index = all_games.index(the_game)

        table = the_game.table
        # making a table embed
        table_embed = discord.Embed(color = 0xb2a2c2, title = "┳━┳ Table ┳━┳")
        for person in table:
            showable = show(table[person])
            
            if type(person) == str:
                table_embed.add_field(name = "Nobody", value = showable)
                continue
            table_embed.add_field(name = bot.get_user(person).name, value = showable)
        # saving the table embed
        await ctx.send(embed = table_embed)

## **Beginning of the mafia game**

In [5]:
class mafia_game:
    def __init__(self, member_list, orig_channel, start_time):
        self.member_list = member_list
        self.orig_channel = orig_channel
        self.start_time =  start_time
        self.active_roles = []
    
    def give_roles(self):
        assigned = dict()
        members = self.member_list
        member_count = len(members)
        
        # one mafiosi per four people (meaning there is a minimum of four people)
        mafia_role = {"mafiosi":'votes out a villager victim each "night"'}
        roles = [mafia_role]*int(member_count/4)
        
        # adding extra roles if there are more than eight people
        mason_role = {"mason":"knows the other mason"}
        if member_count >= 8:
            extra_roles = [{"seer":"votes in the nighttime poll, but only to see if a person is mafia or not"},
                           {"protector/doctor":"protects a person from the mafia's wrath each night"},] + [mason_role]*2
            # making the total roles: (at least) two mafiosi, four misc roles, and (at least) 2 villagers
            roles += extra_roles
        roles += [{"villager":"try and catch the mafia members"}]*int(len(members)-len(roles)) # making up the difference with villagers
        
        # saving the playing roles to a list
        self.active_roles += roles 
        
        # assigning roles to members
        for member in members:
            # picking a random role
            rand_role = np.random.choice(roles)
            try:
                # making sure mafian is the person's only roles
                if mafia_role in assigned[member]: 
                    continue 
                # if they have other roles, don't add a mafia role
                elif assigned[member] and rand_role == mafia_role: 
                    continue
                # so someone isn't both masons
                elif mason_role in assigned[member] and rand_role == mason_role: 
                    continue
                else:
                    assigned[member].append(rand_role)
            except KeyError:
                assigned[member] = [rand_role] # defining their role
            roles.remove(rand_role)
        self.member_list = assigned
        return
    
    def remove_player(self, player):
        # member_list is a dict, so the player is their index
        self.member_list.pop(player)
        return
    
    def player_role(self, player):
        # getting a player's roles
        player_roles = self.member_list[player]
        return player_roles 
    
    def role_player(self, role):
        # getting the role's players
        member_list = self.member_list
        # evey member with the role in their role list
        role_members = [member for member in member_list if role in [' '.join(role) for role in member_list[member]]]
        return role_members
    
    def win_check(self):
        # the first role name for each member's role list
        member_roles = [list(role[0])[0] for role in list(self.member_list.values())]
        
        vc = len(['v' for role in member_roles if role != "mafiosi"]) # villager count
        mc = len(member_roles) - vc # mafia count
        
        if mc >= vc:
            return "mw" # mafia win
        elif not mc:
            return "vw" # villager win
        else:
            return "sp" # still playing
        
    def add_player(self, player):
        # giving the new player a role to the same chance as everyone else had
        chance = random.randint(0, len(self.member_list)+1)
        if chance in range(int((len(self.member_list)+1)/4)):
            self.member_list[player] = {"mafiosi":'votes out a villager victim each "night"'}
        else:
            self.member_list[player] = {"villager":"try and catch the mafia members"}
        
    def play_time(self):
        # the number of minutes since the game started
        now = datetime.now()
        gametime = (now - self.start_time) / timedelta(minutes = 1)
        return gametime

In [6]:
class play_mafia_game(commands.Cog):
    def __init__(self, bot):
        self.bot = bot
        
    @commands.command(name = 'mention_mafia_game', aliases = ['mmg'])
    async def mention_playing(self, ctx):
        """**Start a mafia game with whoever you mention (bonus roles if you have 8+ players!)**"""
        # making sure the game is being started from a guild
        if isinstance(ctx.channel, discord.DMChannel):
            await ctx.send("Start a game from a guild, silly goose! <:goose_joker:738196077047578695>")
            return
        
        # loading the list of game classes
        with open("mafia_games.pkl", 'rb') as f:
            games = pkl.load(f) 
            
        # a list of mentioned users' ids
        players = [int(user.id) for user in ctx.channel.members if user.mentioned_in(ctx.message) and not user.bot]
        
        # adding the author's id to the game if they weren't already there
        if not ctx.author.id in players:
            players.append(int(ctx.author.id))
        
        # repeating users if the minimum number of players isn't fulfilled
        while len(players) < 4: # the minimum number of players is four
            rand_p = np.random.choice(players)
            players.append(rand_p)
            
        # list of existing player id lists
        preexisting_players = [game.member_list for game in games]
        
        # checking if any of the new players are in the old lists
        check = check_for_repeats(new_list = players, old_lists = preexisting_players) # returns [[repeaters], [lists with repeaters]]
        if check[0]: 
            # calling out the repeaters
            for user in check[0]:
                user = bot.get_user(user)
                await ctx.send(f"**{user.mention}** is already in a mafia game :pensive:")
            await ctx.send("Only one mafia game at a time per person, silly goose! <:goose_joker:738196077047578695>")
        # when no game with people in the new players was found
        else:
            game_start_time = datetime.now()
            orig_channel = int(ctx.channel.id)
            
            # initializing a mafia game
            mg = mafia_game(players, orig_channel, game_start_time)
            mg.give_roles()
            
            # saving the game 
            games.append(mg) 
            with open("mafia_games.pkl", 'wb') as f:
                pkl.dump(games, f)
                
            # sending the active roles in an embed
            embed = discord.Embed(color = 0xb2a2c2, title = "Active roles")
            active_roles = mg.active_roles
            for role in active_roles:
                role_name = ' '.join(role.keys())
                embed.add_field(name = role_name, value = role[role_name])
            await ctx.send(embed = embed)
            
            # sending each player their roles in an embed
            for player_id in mg.member_list:
                player_role = mg.player_role(player_id)
                
                player = bot.get_user(player_id)
                embed = discord.Embed(color = 0xb2a2c2, 
                                      title = "Your role(s)")
                for role in player_role:
                    role_name = ' '.join(role.keys())
                    embed.add_field(name = role_name, value = role[role_name])
                await player.send(embed = embed)
                
            # telling each masoner their mason buddy
            masons = mg.role_player("mason") # role => players
            for i, player_id in enumerate(masons):
                mason_buddy = bot.get_user(masons[i-1])
                mason = bot.get_user(player_id)
                
                mason_embed = discord.Embed(color = 0xb2a2c2, description = "Your mason buddy is: " + mason_buddy.mention)
                await mason.send(embed = mason_embed)
            
            # telling each mafian their mafia buddies
            mafians = [bot.get_user(user_id) for user_id in mg.role_player("mafiosi")]
            for user in mafians:
                mafia_embed = discord.Embed(color = 0xb2a2c2, 
                                            title = 'All the "mafia members"', 
                                            description = ", ".join([player.mention for player in mafians]))
                await user.send(embed = mafia_embed)
    
    @commands.command(name = 'voice_mafia_game', aliases = ['vmg'])
    async def voice_start_playing(self, ctx):
        """**Start a mafia game with whoever you're in a voice channel with (bonus roles if you have 8+ players!)**"""
        # making sure the game is being started from a guild
        if isinstance(ctx.channel, discord.DMChannel):
            await ctx.send("Start a game from a guild, silly goose! <:goose_joker:738196077047578695>")
            return
        
        # loading the list of game classes
        with open("mafia_games.pkl", 'rb') as f:
            games = pkl.load(f) 
            
        # a list of the voice members
        try:
            voice_users = ctx.author.voice.channel.members
            players = [int(user.id) for user in voice_users if not user.bot]
        except AttributeError:
            await ctx.send("You need to be in a voice channel to start a mafia with its members, silly goose! <:goose_joker:738196077047578695>")
            return
        
        # repeating users if the minimum number of players isn't fulfilled
        while len(players) < 4: # the minimum number of players is four
            rand_p = np.random.choice(players)
            players.append(rand_p)
            
        # list of existing player id lists
        preexisting_players = [game.member_list for game in games]
        
        # checking if any of the new players are in the old lists
        check = check_for_repeats(new_list = players, old_lists = preexisting_players) # returns [[repeaters], [lists with repeaters]]
        if check[0]: 
            # calling out the repeaters
            for user in check[0]:
                user = bot.get_user(user)
                await ctx.send(f"**{user.mention}** is already in a mafia game :pensive:")
            await ctx.send("Only one mafia game at a time per person, silly goose! <:goose_joker:738196077047578695>")
        # when no game with people in the new players was found
        else:
            game_start_time = datetime.now()
            orig_channel = int(ctx.channel.id)
            
            # initializing a mafia game
            mg = mafia_game(players, orig_channel, game_start_time)
            mg.give_roles()
            
            # saving the game 
            games.append(mg) 
            with open("mafia_games.pkl", 'wb') as f:
                pkl.dump(games, f)
                
            # sending the active roles in an embed
            embed = discord.Embed(color = 0xb2a2c2, title = "Active roles")
            active_roles = mg.active_roles
            for role in active_roles:
                role_name = ' '.join(role.keys())
                embed.add_field(name = role_name, value = role[role_name])
            await ctx.send(embed = embed)
            
            # sending each player their roles in an embed
            for player_id in mg.member_list:
                player_role = mg.player_role(player_id)
                
                player = bot.get_user(player_id)
                embed = discord.Embed(color = 0xb2a2c2, 
                                      title = "Your role(s)")
                for role in player_role:
                    role_name = ' '.join(role.keys())
                    embed.add_field(name = role_name, value = role[role_name])
                await player.send(embed = embed)
                
            # telling each masoner their mason buddy
            masons = mg.role_player("mason") # role => players
            for i, player_id in enumerate(masons):
                mason_buddy = bot.get_user(masons[i-1])
                mason = bot.get_user(player_id)
                
                mason_embed = discord.Embed(color = 0xb2a2c2, description = "Your mason buddy is: " + mason_buddy.mention)
                await mason.send(embed = mason_embed)
            
            # telling each mafian their mafia buddies
            mafians = [bot.get_user(user_id) for user_id in mg.role_player("mafiosi")]
            for user in mafians:
                mafia_embed = discord.Embed(color = 0xb2a2c2, 
                                            title = 'All the "mafia members"', 
                                            description = ", ".join([player.mention for player in mafians]))
                await user.send(embed = mafia_embed)
                
    @commands.command(name = 'react_mafia_game', aliases = ['rmg'])
    async def react_start_playing(self, ctx):
        """**Start a mafia game with whoever reacts to my message (bonus roles if you have 8+ players!)**"""
        # making sure the game is being started from a guild
        if isinstance(ctx.channel, discord.DMChannel):
            await ctx.send("Start a game from a guild, silly goose! <:goose_joker:738196077047578695>")
            return
        
        # loading the list of game classes
        with open("mafia_games.pkl", 'rb') as f:
            games = pkl.load(f) 
            
        # returning true when a non-bot reaction was added to the poll message
        poll = discord.Embed(color = 0xb2a2c2, description = "React to this message with 🎫 if you want to join the mafia game")
        poll.set_footer(text = "Poll times out after 20 seconds of inactivity",
                        icon_url = 'https://media.discordapp.net/attachments/720681623577690176/734896506917617664/joker_card_3.png?width=919&height=919')
        
        # sending the poll and adding a reaction to it
        poll_msg = await ctx.send(embed = poll)
        await poll_msg.add_reaction('🎫')
        
        # making sure we're only resetting the timer for the right reaction
        def check(reaction, user):
            return reaction.message.id == poll_msg.id and not user.bot and reaction.emoji == '🎫'

        # while we don't have enough reactions or there's inactivity
        while True:
            try:
                # checking for reactions unless twenty seconds pass
                reaction, user = await bot.wait_for('reaction_add', check = check, timeout = 20)
            except asyncio.TimeoutError:
                break
        
        # getting the people who reacted with ticket
        players = []
        poll_msg = await ctx.channel.fetch_message(poll_msg.id)
        for reaction in poll_msg.reactions:
            if reaction.emoji == '🎫':
                users = await reaction.users().flatten()
                players = [user.id for user in users if not user.bot]
        
        # returning if there are no reactions
        if not players:
            await ctx.send("All poll, no reacts :("
                           + "\nSilly gooses! <:goose_joker:738196077047578695>")
            return
        
        # repeating users if the minimum number of players isn't fulfilled
        while len(players) < 4: # the minimum number of players is four
            rand_p = np.random.choice(players)
            players.append(rand_p)
            
        # list of existing player id lists
        preexisting_players = [game.member_list for game in games]
        
        # checking if any of the new players are in the old lists
        check = check_for_repeats(new_list = players, old_lists = preexisting_players) # returns [[repeaters], [lists with repeaters]]
        if check[0]: 
            # calling out the repeaters
            for user in check[0]:
                user = bot.get_user(user)
                await ctx.send(f"**{user.mention}** is already in a mafia game :pensive:")
            await ctx.send("Only one mafia game at a time per person, silly goose! <:goose_joker:738196077047578695>")
        # when no game with people in the new players was found
        else:
            game_start_time = datetime.now()
            orig_channel = int(ctx.channel.id)
            
            # initializing a mafia game
            mg = mafia_game(players, orig_channel, game_start_time)
            mg.give_roles()
            
            # saving the game 
            games.append(mg) 
            with open("mafia_games.pkl", 'wb') as f:
                pkl.dump(games, f)
                
            # sending the active roles in an embed
            embed = discord.Embed(color = 0xb2a2c2, title = "Active roles")
            active_roles = mg.active_roles
            for role in active_roles:
                role_name = ' '.join(role.keys())
                embed.add_field(name = role_name, value = role[role_name])
            await ctx.send(embed = embed)
            
            # sending each player their roles in an embed
            for player_id in mg.member_list:
                player_role = mg.player_role(player_id)
                
                player = bot.get_user(player_id)
                embed = discord.Embed(color = 0xb2a2c2, 
                                      title = "Your role(s)")
                for role in player_role:
                    role_name = ' '.join(role.keys())
                    embed.add_field(name = role_name, value = role[role_name])
                await player.send(embed = embed)
                
            # telling each masoner their mason buddy
            masons = mg.role_player("mason") # role => players
            for i, player_id in enumerate(masons):
                mason_buddy = bot.get_user(masons[i-1])
                mason = bot.get_user(player_id)
                
                mason_embed = discord.Embed(color = 0xb2a2c2, description = "Your mason buddy is: " + mason_buddy.mention)
                await mason.send(embed = mason_embed)
            
            # telling each mafian their mafia buddies
            mafians = [bot.get_user(user_id) for user_id in mg.role_player("mafiosi")]
            for user in mafians:
                mafia_embed = discord.Embed(color = 0xb2a2c2, 
                                            title = 'All the "mafia members"', 
                                            description = ", ".join([player.mention for player in mafians]))
                await user.send(embed = mafia_embed)
 
    @commands.command(name = "nighttime", aliases = ['night_poll', 'np'])
    async def night_game(self, ctx):
        """Triggers the nighttime poll"""
        async with ctx.typing():
            game_things = get_mafia_game(ctx.author.id)
            if game_things:
                the_game = game_things[0]
                game_members = the_game.member_list
                
                # saving all the games in a variable for updating the game class
                all_games = game_things[1]
                game_index = all_games.index(the_game) 
                
                # making sure polling is from the original channel
                if ctx.channel.id != the_game.orig_channel:
                    await ctx.send("Run the poll from the game's channel, silly goose! <:goose_joker:738196077047578695>")
                    return
                
                # making a poll embed
                poll = discord.Embed(color = 0xb2a2c2,
                                     title = "✧Nighttime poll✧", 
                                     description = "Mafia people, click on the emoji of the people you want out" 
                                     + "\nDoctors and seers, click on the emoji of the person you want to affect")
                poll.set_footer(text = "Poll times out after 20 seconds of inactivity", 
                                icon_url = 'https://media.discordapp.net/attachments/720681623577690176/734896506917617664/joker_card_3.png?width=919&height=919')
                
                # the list of emojis to choose from
                emojis = ['😁', '😅', '🤣', '🙂', '🙃', '😉', '😊', '😇', '😍', '🤩', '😋', '🤪', '🤑', '🤗', '🤭', '🤨', '😑', '😶',
                          '😏', '😒', '🙄', '😬', '🤥', '😌', '😔', '😪', '🤤', '😴', '🤕', '🤧', '🥵', '🥶', '🥴', '🤯', '🤠', '🥳',
                          '😎', '🤓', '🧐', '😕', '😟', '🙁', '😲', '😳', '🥺', '😧', '😨', '😢', '😭', '😱', '😖', '😞', '😩',]
                
                # initializing a dict (will store each emoji to its player)
                emojis_players = {}
                
                # assigning each player an emoji
                for player in game_members:
                    # picking a random emoji from the list and taking it from the list
                    reaction_emoji = np.random.choice(emojis)
                    emojis.remove(reaction_emoji)
                    
                    # assigning the emoji a player
                    emojis_players[reaction_emoji] = player # emoji : player_id
                    
                    # adding the member and their emoji to the embed
                    member = bot.get_user(player)
                    poll.add_field(name = member.name, value = reaction_emoji)
                
                # sending the poll embed 
                poll_msg = await ctx.send(embed = poll)
                
                # adding the reactions to the message
                react_emojis = list(emojis_players)
                for emoji in react_emojis:
                    await poll_msg.add_reaction(emoji)
                
                # all the people to each role we need (ids)
                masons = the_game.role_player("mason")
                villagers = the_game.role_player("villager") + masons
                doctor_id = the_game.role_player("protector/doctor")
                seer_id = the_game.role_player("seer")
                voting_people = [person for person in game_members if not person in villagers]
                
                mafia_vote = {} # mafian : their voted out person
                protected = 0 # the protected person's id
                seen = 0 # seen could be updated to a user id 
                
                # returning true when a non-bot reaction was added to the poll message
                def check(reaction, user):
                    return reaction.message.id == poll_msg.id and not user.bot
                
                # looping the check for reactions while there aren't enough reactions, or twenty seconds of inactivity
                while voting_people:
                    try:
                        # checking for reactions
                        reaction, user = await bot.wait_for('reaction_add', check = check, timeout = 20)
                        
                        # removing the reaction
                        await reaction.remove(user)
                        
                        user_id = user.id
                        react_emoji = reaction.emoji
                        
                        # checking if the user can vote
                        if user_id in voting_people and react_emoji in react_emojis:
                            # mafian_id : their voted person
                            mafia_vote[user_id] = emojis_players[react_emoji]
                            
                            # making sure people can only vote once
                            voting_people.remove(user_id)
                        # checking if the person is the doctor
                        elif user_id in doctor_id: 
                            # updating protected 
                            protected = emojis_players[react_emoji]
                            
                            # making sure the doctor can only protect one person
                            doctor_id.remove(user_id)
                        # checking if the user is the seer
                        elif user_id in seer_id:
                            # updating the value of seen
                            seen = emojis_players[react_emoji]
                            
                            # making sure the seer can only see one person's roles
                            seer_id.remove(user_id)
                        # ignoring everyone else's reactions (mafians)
                        else:
                            continue
                    # leaving the loop if the timeout is reached
                    except asyncio.TimeoutError:
                        break
                await poll_msg.clear_reactions() # we don't need more voters, this signals the vote is over
                
                # finding the most-voted person
                try:
                    voted = stats.mode(list(mafia_vote.values()))
                except stats.StatisticsError:
                    await ctx.send("No votes :/")
                    return
                
                # checking if the voted person is in the protected people list
                if voted != protected:
                    # removing the player
                    the_game.remove_player(voted)
                    voted_member = bot.get_user(voted)
                    
                    # a list of responses to pick from
                    responses = [" was removed by the mafia :pensive:", 
                                 " was sniped", 
                                 " got knocked out in the night", 
                                 " was deemed to much of a risk to keep alive..."]
                    # picking a random response
                    response = np.random.choice(responses)
                    
                    # notifying people of the verdict
                    await ctx.send(voted_member.mention + response)
                    
                    # checking if there was a win
                    game_status = the_game.win_check()
                    if game_status == "mw":
                        await ctx.send(":fire: :sunglasses: The mafia has won :sunglasses: :fire:")
                        
                        # removing the game
                        all_games.remove(the_game)
                        with open("mafia_games.pkl", 'wb') as f:
                            pkl.dump(all_games, f)
                    elif game_status == "vw":
                        await ctx.send(":tada: The villagers have won :tada:")
                        
                        # removing the game
                        all_games.remove(the_game)
                        with open("mafia_games.pkl", 'wb') as f:
                            pkl.dump(all_games, f)
                    else:
                        await ctx.send(":sunny: Daytime now!")
                        
                        # saving the game
                        all_games[game_index] = the_game
                        with open("mafia_games.pkl", 'wb') as f:
                            pkl.dump(all_games, f)
                # when the player was protected
                else:
                    await ctx.send("The mafia was thwarted")

                if seen:
                    seer = bot.get_user(seer_id[0]) # seer_id is a list of length one
                    seen_member = bot.get_user(seen) # getting the member from their id
                    
                    # checking if the person is in the mafia
                    mafians = the_game.role_player("mafiosi")
                    if_mafia = bool(seen in mafians)
                    
                    # making an embed to notify the seer
                    if if_mafia:
                        seer_embed = discord.Embed(color = 0xb2a2c2, title = if_mafia, description = seen_member.mention + " is in the mafia")
                    else:
                        seer_embed = discord.Embed(color = 0xb2a2c2, title = if_mafia, description = seen_member.mention + " is not in the mafia")
                    # sending the embed to the seer
                    await seer.send(embed = seer_embed)
            else:
                await ctx.send("You're not playing a mafia game, silly goose! <:goose_joker:738196077047578695>")
                
    @commands.command(name = "daytime", aliases = ['day_poll', 'dp'])
    async def day_game(self, ctx):
        """Triggers the daytime poll"""
        async with ctx.typing():
            game_things = get_mafia_game(ctx.author.id)
            if game_things:
                the_game = game_things[0]
                game_members = the_game.member_list
                
                # saving all the games in a variable for updating the game class
                all_games = game_things[1]
                game_index = all_games.index(the_game) 
                
                # making sure polling is from the original channel
                if ctx.channel.id != the_game.orig_channel:
                    await ctx.send("Run the poll from the game's channel, silly goose! <:goose_joker:738196077047578695>")
                    return
                
                # making the daytime poll embed
                poll = discord.Embed(color = 0xb2a2c2,
                                     title = "✦Daytime poll✦", 
                                     description = "Villagers, click on the emoji of the person you suspect within the next two minutes")
                poll.set_footer(text = "Poll times out after 20 seconds of inactivity", 
                                icon_url = 'https://media.discordapp.net/attachments/720681623577690176/734896506917617664/joker_card_3.png?width=919&height=919')
                
                # the list of people emojis
                emojis = ['😁', '😅', '🤣', '🙂', '🙃', '😉', '😊', '😇', '😍', '🤩', '😋', '🤪', '🤑', '🤗', '🤭', '🤨', '😑', '😶',
                          '😏', '😒', '🙄', '😬', '🤥', '😌', '😔', '😪', '🤤', '😴', '🤕', '🤧', '🥵', '🥶', '🥴', '🤯', '🤠', '🥳',
                          '😎', '🤓', '🧐', '😕', '😟', '🙁', '😲', '😳', '🥺', '😧', '😨', '😢', '😭', '😱', '😖', '😞', '😩',]
                
                # emoji : player id
                emojis_players = {}
                
                # assigning each player an emoji
                for player in game_members:
                    # picking a random emoji from the list and taking it from the list
                    reaction_emoji = np.random.choice(emojis)
                    emojis.remove(reaction_emoji)
                    
                    # assigning the emoji a player
                    emojis_players[reaction_emoji] = player # emoji : player_id
                    
                    # adding the member and their emoji to the embed
                    member = bot.get_user(player)
                    poll.add_field(name = member.name, value = reaction_emoji)
                
                # sending the poll embed 
                poll_msg = await ctx.send(embed = poll)
                
                # adding the reactions to the message
                react_emojis = list(emojis_players)
                for emoji in react_emojis:
                    await poll_msg.add_reaction(emoji)
                
                village_vote = {} # villager : voted out person
                mafia_members = the_game.role_player("mafiosi")
                voting_people = [person for person in game_members if not person in mafia_members]
                
                # returning true when a non-bot reaction was added to the poll message
                def check(reaction, user):
                    return reaction.message.id == poll_msg.id and not user.bot
                
                # while we don't have enough reactions or there's inactivity
                while voting_people:
                    try:
                        # checking for reactions unless 20 seconds
                        reaction, user = await bot.wait_for('reaction_add', check = check, timeout = 20)
                        
                        # removing the new reaction
                        await reaction.remove(user)
                        
                        user_id = user.id
                        react_emoji = reaction.emoji
                        
                        # checking if the user can vote and if the reaction is applicable
                        if user_id in voting_people and react_emoji in react_emojis:
                            # villager : voted person
                            village_vote[user_id] = emojis_players[react_emoji]
                            
                            # making sure the person can't vote twice
                            voting_people.remove(user_id)
                        else:
                            continue
                    except asyncio.TimeoutError:
                        break
                
                # signaling we don't need more reactions by clearing all the reactions
                await poll_msg.clear_reactions() 
                
                # finding the most-voted person
                try:
                    voted = stats.mode(list(village_vote.values()))
                except stats.StatisticsError:
                    await ctx.send("No votes :/")
                    return
                
                the_game.remove_player(voted)
                
                # notifying the person was removed
                voted_member = bot.get_user(voted)
                responses = [" was removed by the villagers :pensive:", 
                             " was pitchforked", 
                             " was deemed to suspicious..."]
                # sending a random response
                response = np.random.choice(responses)
                await ctx.send(voted_member.mention + response)
                
                # checking for wins
                game_status = the_game.win_check()
                if game_status == "mw":
                    await ctx.send(":fire: :sunglasses: The mafia has won :sunglasses: :fire:")
                    
                    # removing the game
                    all_games.remove(the_game)
                    with open("mafia_games.pkl", 'wb') as f:
                        pkl.dump(all_games, f)
                elif game_status == "vw":
                    await ctx.send(":tada: The villagers have won :tada:")
                    
                    # removing the game
                    all_games.remove(the_game)
                    with open("mafia_games.pkl", 'wb') as f:
                        pkl.dump(all_games, f)
                else:
                    await ctx.send("✧ Nighttime now! ✧")
                    
                    # saving the game
                    all_games[game_index] = the_game
                    with open("mafia_games.pkl", 'wb') as f:
                        pkl.dump(all_games, f)
            else:
                await ctx.send("You're not playing a mafia game, silly goose! <:goose_joker:738196077047578695>")
                
    @commands.command(name = "leave_mafia_game", aliases = ["lmg", "ml", "lm"])
    async def leaving_the_mafia(self, ctx):
        """You leave the mafia game (works in any channel or if you DM me)"""
        async with ctx.typing():
            game_things = get_mafia_game(ctx.author.id)
            if game_things:
                the_game = game_things[0]
                
                # saving for updating the game class 
                all_games = game_things[1]
                game_index = all_games.index(the_game)
                
                the_game.remove_player(ctx.author.id)
                
                # saying notifying about the leave
                await ctx.author.send("Bye, silly goose! <:goose_joker:738196077047578695>")
                orig_channel = bot.get_channel(the_game.orig_channel)
                await orig_channel.send(f"If you didn't already know, {ctx.author.name} has abandoned us :pensive:")
                
                # checking for a win
                game_status = the_game.win_check()
                orig_channel = bot.get_channel(the_game.orig_channel)
                if game_status == "mw":
                    await orig_channel.send(":fire: :sunglasses: The mafia has won :sunglasses: :fire:")
                    
                    # removing the game
                    all_games.remove(the_game)
                    with open("mafia_games.pkl", 'wb') as f:
                        pkl.dump(all_games, f)
                elif game_status == "vw":
                    await orig_channel.send(":tada: The villagers have won :tada:")
                    
                    # removing the game
                    all_games.remove(the_game)
                    with open("mafia_games.pkl", 'wb') as f:
                        pkl.dump(all_games, f)
                else:
                    # saving the game
                    all_games[game_index] = the_game
                    with open("mafia_games.pkl", 'wb') as f:
                        pkl.dump(all_games, f)
            else:
                await ctx.send("You're not playing a mafia game, silly goose! <:goose_joker:738196077047578695>")
                
    @commands.command(name = "join_mafia_game", aliases = ["jmg"])
    async def joing_the_mafia(self, ctx):
        """Join the mafia game of the person you mention"""
        async with ctx.typing():
            # the first person mentioned if there are mentioned people
            members = ctx.channel.members
            mentioned = [user for user in members if user.mentioned_in(ctx.message) and not user.bot]
            if mentioned:
                game_player = mentioned[0] 
            else:
                await ctx.send("Mention a person in a game to join their game, silly goose! <:goose_joker:738196077047578695>")
                return
            
            game_things = get_mafia_game(game_player.id)
            if game_things:
                the_game = game_things[0]
                
                # saving this for updating the game save later
                all_games = game_things[1]
                game_index = all_games.index(the_game)
                
                # making sure the joiner has access to the game channel by limiting where they can join
                if ctx.channel.id != the_game.orig_channel:
                    orig_channel = bot.get_channel(the_game.orig_channel)
                    await ctx.send(f"Join {game_player.name}'s game from {orig_channel.mention}, silly goose! <:goose_joker:738196077047578695>")
                
                # making sure the author isn't already in the game
                author_id = ctx.author.id
                if author_id not in the_game.member_list:
                    the_game.add_player(author_id)
                    
                    # saving the game
                    all_games[game_index] = the_game
                    with open("mafia_games.pkl", 'wb') as f:
                        pkl.dump(all_games, f)
                    
                    player_role = the_game.player_role(author_id) 
                    
                    # making an embed to send the person their roles
                    embed = discord.Embed(color = 0xb2a2c2, 
                                          title = "Your role")
                    role_name = ' '.join(list(player_role.keys()))
                    embed.add_field(name = role_name, value = player_role[role_name]) # the role desc
                    # sending the role embed
                    await ctx.author.send(embed = embed)
                else:
                    await ctx.send("You're already in that game, silly goose! <:goose_joker:738196077047578695>")
            else:
                await ctx.send("The first person mentioned wasn't in a game, silly goose! <:goose_joker:738196077047578695>")
                
    @commands.command(name = 'mafia_play_time', aliases = ['mpt', 'mgt'])
    async def gameplay_time(self, ctx):
        """How long you've been playing mafia"""
        async with ctx.typing():
            game_things = get_mafia_game(ctx.author.id)
            if game_things:
                the_game = game_things[0]
                gameplay_time = the_game.play_time()
                
                # making an embed to show the time in minutes (not seconds because that would be weird)
                embed = discord.Embed(color = 0xb2a2c2)
                embed.add_field(name = "Time playing:", value = f"{int(gameplay_time)} minutes and {int(round(60*(gameplay_time%1), 0))} seconds")
                await ctx.send(embed = embed)
            else:
                await ctx.send("You're not playing a mafia game, silly goose! <:goose_joker:738196077047578695>")

In [7]:
class bot_owner_commands(commands.Cog):
    def __init__(self, bot):
        self.bot = bot
        
    @commands.command(name = 'online', aliases = ['on','o'])
    async def hiii(self, ctx): # technically not an exclusive command, but it's really only useful for me
        await ctx.send('Present!<:ace_goose:734497949542908034>')
                
    @commands.command(name = "old_game_purge", aliases = ['ogpurge'])
    async def purging_old_games(self, ctx):
        owner = await bot.is_owner(ctx.author)
        if not owner: # we don't want random people purging old games
            ctx.send("This command is not for you, silly goose!<:ace_goose:734497949542908034>")
            return
        
        # getting all the mafia game classes and all the card game classes
        with open("mafia_games.pkl", 'rb') as f:
            mafia_game_classes = pkl.load(f) # loading the list of game classes
        with open("card_games.pkl", 'rb') as f:
            card_game_classes = pkl.load(f) # loading the list of game classes
            
        total_classes = len(mafia_game_classes) + len(card_game_classes)
        removed_classes = 0

        if card_game_classes:
            for cclass in card_game_classes:
                if cclass.play_time() > 1440: # the number of minutes in a day
                    # notifying the original channel their game was removed
                    orig_channel = bot.get_channel(cclass.orig_channel)
                    await orig_channel.send("No games for over a day, silly gooses! <:goose_joker:738196077047578695>")
                    card_game_classes.remove(cclass)
                    removed_classes += 1
            if removed_classes: # updating if there were any removed
                with open("card_games.pkl", 'wb') as f:
                    pkl.dump(card_game_classes, f)
        if mafia_game_classes:
            for mclass in mafia_game_classes:
                if mclass.play_time() > 1440: # the number of minutes in a day
                    # notifying the original channel their game was removed
                    orig_channel = bot.get_channel(mclass.orig_channel)
                    await orig_channel.send("No games for over a day, silly gooses! <:goose_joker:738196077047578695>")
                    mafia_game_classes.remove(mclass)
                    removed_classes += 1
            if removed_classes: # only updating if there were any removed classes
                with open("mafia_games.pkl", 'wb') as f:
                    pkl.dump(mafia_game_classes, f)
        
        # notifying the ctx channel of how many classes were removed
        await ctx.channel.send(f"{removed_classes} game classes were purged from the {total_classes} classes this cycle, master")

In [8]:
class rando_commandos(commands.Cog): # class for testing things with the bot without changing preexisting commands
    def __init__(self, bot):
        self.bot = bot
        
    @commands.command(name = "space", aliases = ["sky"])
    async def generate_night_sky(self, ctx):
        """Displays a clear night sky"""
        message = ''
        space = ['✧', '*', '˚', '.', ',', '✦', '♢', "'"] + ['\n']*2 + [' ']*150
        
        while len(message) < 3000: 
            message += np.random.choice(space)
            
        spaceship_spot = random.randrange(1000, len(message))
        message = message[:spaceship_spot] + '🚀' + message[spaceship_spot:]
        
        # 10% chance to get a shooting star
        if not random.randint(0, 10): 
            comet_spot = random.randrange(1000, len(message)-1) # -1 so the comet isn't put at the end and cut off
            message = message[:comet_spot] + ':comet:' + message[comet_spot:]
        
        # sectioning the sky into parts of sendable length
        for sendable in [message[i:i+1500] for i in range(0, len(message), 1500)]:
            if len(sendable) < 1500:
                break # no weird little messages
            await ctx.send(sendable)
            
    @commands.command(name = 'add_gamEd', aliases = ['ad', 'ge'])
    async def add_gamEd(self, ctx):
        """Invite me to a server, silly goose! <:goose_joker:738196077047578695>"""
        embed = discord.Embed(title = "Invite me, silly goose! <:goose_joker:738196077047578695>", 
                              url = "https://discord.com/api/oauth2/authorize?client_id=700167135288098876&permissions=470150259&scope=bot",
                              description = "I do discord game things",
                              color = 0xb2a2c2)
        embed.set_thumbnail(url = 'https://media.discordapp.net/attachments/720681623577690176/734896506917617664/joker_card_3.png?width=919&height=919')
        await ctx.send(embed = embed)
            
    @commands.command(name = "card_help", aliases = ['csos', 'chelp'])
    async def card_help(self, ctx):
        """Help with playing cards"""
        play_cards = bot.get_cog('play_cards')
        card_commands = play_cards.get_commands()
        
        if isinstance(ctx.channel, discord.DMChannel):
            commands_embed = discord.Embed(title = 'Complete command list',
                                           color = 0xb2a2c2, 
                                           description = "")
            commands_embed.set_footer(text = "There's a more detailed help command in guild channels :)", 
                                      icon_url = 'https://media.discordapp.net/attachments/720681623577690176/734896506917617664/joker_card_3.png?width=919&height=919')
            # simply adding command names
            for command in card_commands:
                commands_embed.description += f"\n**{command.name} **\nAKA: `{'` | `'.join(command.aliases)}`"
            await ctx.send(embed = commands_embed)
            return               
        
        # setting up an embed for the "table of contents"
        toc_embed = discord.Embed(title = 'Table of contents for card help',
                                  color = 0xb2a2c2,
                                  description = "Prefixes: `>>` | `<<` | `>.<` | `<>` | `-->`"
                                  + "\n**Reactions to pages:**"
                                  + "\n0️⃣ `Brings you back to the table of contents`"
                                  + "\n1️⃣ `Game setup commands`"
                                  + "\n2️⃣ `General in-game commands`"
                                  + "\n3️⃣ `Card-playing commands`"
                                  + "\n4️⃣ `Tablish commands`")
        toc_embed.set_thumbnail(url = 'https://media.discordapp.net/attachments/720681623577690176/734896506917617664/joker_card_3.png?width=919&height=919')
        toc_embed.set_footer(text = "Page functionality times out after 2 minutes of inactivity")
        
        # setting up an embed for game setup
        game_setup_commands_embed = discord.Embed(title = 'Card game setup commands help',
                                                  color = 0xb2a2c2,
                                                  description = "React with 0️⃣ to go back to the table of contents"
                                                  + "\nPrefixes: `>>` | `<<` | `>.<` | `<>` | `-->`")
        game_setup_commands_embed.set_thumbnail(url = 'https://media.discordapp.net/attachments/720681623577690176/734896506917617664/joker_card_3.png?width=919&height=919')
        game_setup_commands_embed.set_footer(text = "Page functionality times out after 2 minutes of inactivity")
        
        # setting up an embed for in-game commands
        in_game_commands_embed = discord.Embed(title = 'In-game commands help', 
                                               color = 0xb2a2c2, 
                                               description = "React with 0️⃣ to go back to the table of contents"
                                                  + "\nPrefixes: `>>` | `<<` | `>.<` | `<>` | `-->`")
        in_game_commands_embed.set_thumbnail(url = 'https://media.discordapp.net/attachments/720681623577690176/734896506917617664/joker_card_3.png?width=919&height=919')
        in_game_commands_embed.set_footer(text = "Page functionality times out after 2 minutes of inactivity")
        
        # setting up an embed for playing cards
        card_commands_embed = discord.Embed(title = 'Card-playing commands help', 
                                               color = 0xb2a2c2, 
                                               description = "React with 0️⃣ to go back to the table of contents"
                                                  + "\nPrefixes: `>>` | `<<` | `>.<` | `<>` | `-->`")
        card_commands_embed.set_thumbnail(url = 'https://media.discordapp.net/attachments/720681623577690176/734896506917617664/joker_card_3.png?width=919&height=919')
        card_commands_embed.set_footer(text = "Page functionality times out after 2 minutes of inactivity")
        
        # setting up an embed for table commands
        table_commands_embed = discord.Embed(title = '┳━┳ Table commands help ┳━┳', 
                                               color = 0xb2a2c2, 
                                               description = "React with 0️⃣ to go back to the table of contents"
                                                  + "\nPrefixes: `>>` | `<<` | `>.<` | `<>` | `-->`")
        table_commands_embed.set_thumbnail(url = 'https://media.discordapp.net/attachments/720681623577690176/734896506917617664/joker_card_3.png?width=919&height=919')
        table_commands_embed.set_footer(text = "Page functionality times out after 2 minutes of inactivity")
        
        # naming the different functions for each section
        game_setup_command_names = ['mention_card_game', 'voice_card_game', 'react_card_game', 'shuffle', 'deal_cards_evenly', 'deal_cards_numerically', 'remove_n_cards']
        in_game_command_names = ['my_cards', 'get_stats', 'cards_play_time','reset_deck', 'join_card_game', 'leave_card_game', 'remove_n_cards']
        card_playing_command_names = ['play_card', 'draw_card', 'fish_cards', 'get_cards']
        table_command_names = ['table_put', 'table', 'table_take', 'table_random', 'table_nobody']
        
        # going through every command
        for bot_command in card_commands:
            docs = bot_command.help.strip() # doc strings are at the top of each function
            if bot_command.name in game_setup_command_names: # the game setup commands
                game_setup_commands_embed.add_field(name = bot_command.name, value = 'AKA: `' + '` | `'.join(bot_command.aliases) + '`\n' + docs)
            elif bot_command.name in in_game_command_names: # the in-game commands
                in_game_commands_embed.add_field(name = bot_command.name, value = 'AKA: `' + '` | `'.join(bot_command.aliases) + '`\n' + docs)
            elif bot_command.name in card_playing_command_names: # the card-playing commands
                card_commands_embed.add_field(name = bot_command.name, value = 'AKA: `' + '` | `'.join(bot_command.aliases) + '`\n' + docs)
            elif bot_command.name in table_command_names: # the tablish commands
                table_commands_embed.add_field(name = bot_command.name, value = 'AKA: `' + '` | `'.join(bot_command.aliases) + '`\n' + docs)
            else:
                print(bot_command.name) # to see which ones were missed in naming
        
        # reaction : embed
        react_embed = {'0️⃣' : toc_embed, 
                       '1️⃣' : game_setup_commands_embed, 
                       '2️⃣' : in_game_commands_embed, 
                       '3️⃣' : card_commands_embed, 
                       '4️⃣' : table_commands_embed} 
        
        # starting by sending the TOC
        help_msg = await ctx.send(embed = toc_embed) 
        
        # adding the reactions assigned to each embed
        for reaction in react_embed: 
            await help_msg.add_reaction(reaction)
        
        # returning true if the reaction is applicable to an embed
        help_reacts = list(react_embed)
        def check(reaction, user):
            return reaction.emoji in help_reacts and not user.bot and reaction.message.id == help_msg.id
        
        while True:
            try:
                reaction, user = await bot.wait_for('reaction_add', check = check, timeout = 120)
                await reaction.remove(user)
                await help_msg.edit(embed = react_embed[reaction.emoji]) # editing to the proper embed
            except asyncio.TimeoutError:
                await help_msg.clear_reactions() # ending after two inactive minutes
                break
                
    @commands.command(name = "mafia_help", aliases = ['msos', 'melp', 'mhelp'])
    async def mafia_help(self, ctx):
        """Help with playing mafia"""
        play_mafia = bot.get_cog('play_mafia_game')
        mafia_commands = play_mafia.get_commands()

        game_commands_embed = discord.Embed(title = 'Mafia game help', color = 0xb2a2c2, 
                                            description = "Prefixes: `>>` | `<<` | `>.<` | `<>` | `-->`")
        game_commands_embed.set_thumbnail(url = 'https://media.discordapp.net/attachments/720681623577690176/734896506917617664/joker_card_3.png?width=919&height=919')
        
        # going through the mafia commands
        for bot_command in mafia_commands:
            docs = bot_command.help.strip()
            game_commands_embed.add_field(name = bot_command.name, value = 'AKA: `' + '` | `'.join(bot_command.aliases) + '`\n' + docs)

        help_msg = await ctx.send(embed = game_commands_embed)
    
    @commands.command(name = "help_help", aliases = ['meta-help', 'help', 'SOS', 'hh'])
    async def help_help(self, ctx):
        """Rando commandos"""
        general_commands = rando_commandos.get_commands(self)
        primary_help = discord.Embed(title = "The meta-help", color = 0xb2a2c2,
                                     description = "It's help inception <:goose_joker:738196077047578695>" 
                                     + "\nPrefixes: `>>` | `<<` | `>.<` | `<>` | `-->`")
        primary_help.set_thumbnail(url = 'https://media.discordapp.net/attachments/720681623577690176/734896506917617664/joker_card_3.png?width=919&height=919')

        for bot_command in general_commands:
            docs = bot_command.help.strip()
            primary_help.add_field(name = bot_command.name, value = 'AKA: `' + '` | `'.join(bot_command.aliases) + '`\n' + docs)

        await ctx.send(embed = primary_help)

In [ ]:
bot.add_cog(play_cards(bot))
bot.add_cog(play_mafia_game(bot))
bot.add_cog(rando_commandos(bot))
bot.add_cog(bot_owner_commands(bot))

# status update
@bot.event
async def on_ready():
#     await bot.change_presence(activity = discord.Game(name = 'cards and mafia under the stars | >.<SOS'))
    await bot.change_presence(activity = discord.Activity(type = discord.ActivityType.watching, name = "the stars :) | >.<SOS"))
    guilds = await bot.fetch_guilds(limit = 150).flatten()
    print("We are in", len(guilds), "guilds")
    print('Logged in as: ' + bot.user.name)

# letting the chat know when there's an error 
@bot.event
async def on_command_error(ctx, error):
    if isinstance(error, commands.CommandNotFound):
        await ctx.send("<:ace_goose:734497949542908034>Invalid command, you silly goose!")
    else:
        error_embed = discord.Embed(color = 0xb2a2c2,
                                    title = "Uh-oh",
                                    description = f"```{error}```" 
                                    + "\nYou might have to restart your game")
        error_embed.set_thumbnail(url = 'https://media.discordapp.net/attachments/690624714078683181/743933538939699251/pensive-svg_9.png?width=919&height=919')
        
        # sending the error notif
        await ctx.send(embed = error_embed)
        raise error

# reading the token from a txt file and using it
with open("GamEd_token", mode = 'r') as f:
    token = f.read()
bot.run(token)

We are in 7 guilds
Logged in as: GamEd
